# 集計練習

## やること

- amountに50~-50の間の乱数を足して、最小値を0とした変数を作成
- 日次/週次/月次/年次で集計
  - 普通にとる方法
  - floorでやってみる
  - indexでうまくやってみる
- quantityとamountを可視化
  - matplotlibで普通に可視化
  - plotlyでうまいこと可視化

In [6]:
import numpy as np
import pandas as pd
from IPython.core.display import display

/var/folders/kt/026ybdfj1nx440kk4b1nbgjw0000gn/T/ipykernel_92757/4019042500.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


In [9]:
df = pd.read_csv('data/receipt.csv')
df["amount_randomise"] = df["amount"] + np.random.randint(-50, 50, df.shape[0])
df["amount_randomise"] = df["amount_randomise"].where(df["amount_randomise"] > 0, 0)
display(df.head())
df.describe()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,amount_randomise
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158,124
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81,85
2,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170,155
3,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25,58
4,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90,90


,sales_ymd,sales_epoch,receipt_no,receipt_sub_no,quantity,amount,amount_randomise
count,1.046810e+05,1.046810e+05,104681.000000,104681.000000,104681.000000,104681.000000,104681.000000
mean,2.018063e+07,1.529803e+09,1053.543699,1.500855,1.011282,320.560083,320.123958
std,8.012852e+03,2.580916e+07,298.326621,0.500002,0.114136,477.702749,478.540165
min,2.017010e+07,1.483229e+09,112.000000,1.000000,1.000000,10.000000,0.000000
25%,2.017101e+07,1.507680e+09,1112.000000,1.000000,1.000000,102.000000,109.000000
50%,2.018070e+07,1.530749e+09,1142.000000,2.000000,1.000000,170.000000,172.000000
75%,2.019031e+07,1.552349e+09,1172.000000,2.000000,1.000000,288.000000,291.000000
max,2.019103e+07,1.572480e+09,1664.000000,2.000000,8.000000,10925.000000,10956.000000
